In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
import cv2
import os
import tensorflow as tf
import numpy as np
import pandas as pd
#from google.colab import drive

In [2]:
labels = ['PNEUMONIA', 'NORMAL']
img_size = 150
def get_training_data(data_dir):
    data = []
    for label in labels:
        path = os.path.join(data_dir, label) # junta o diretoria + a "label"
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [3]:


train = get_training_data('train')
test = get_training_data('test')
val = get_training_data('val')


C:\Users\vinni\AppData\Local\Temp\ipykernel_3864\2095957498.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data)


In [4]:
x_train = []
y_train = []

x_val = []
y_val = []

x_test = []
y_test = []

for feature, label in train:
    x_train.append(feature)
    y_train.append(label)

for feature, label in test:
    x_test.append(feature)
    y_test.append(label)

for feature, label in val:
    x_val.append(feature)
    y_val.append(label)

In [5]:
# Normalize the data
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255
x_test = np.array(x_test) / 255

In [6]:
# resize data for deep learning
x_train = x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

x_val = x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)

x_test = x_test.reshape(-1, img_size, img_size, 1)
y_test = np.array(y_test)

In [7]:
# With data augmentation to prevent overfitting and handling the imbalance in dataset

datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range = 0.2, # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip = True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

In [22]:
image_size_ =

150

In [29]:
image_size = x_train.shape[1]
input_shape = (image_size, image_size, 1)


img_input = tf.keras.Input(shape=input_shape)

    #block 1
conv_1 = tf.keras.layers.Conv2D(32 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu', input_shape=(150, 150, 1))(img_input)
norm_1 = tf.keras.layers.BatchNormalization()(conv_1)
maxpool_1 = tf.keras.layers.MaxPool2D((2,2), strides = 2, padding = 'same')(norm_1)

#block 2
conv_2 = tf.keras.layers.Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu')(maxpool_1)
drop_1 = tf.keras.layers.Dropout(0.1)(conv_2)
norm_2 = tf.keras.layers.BatchNormalization()(drop_1)
maxpool_2 = tf.keras.layers.MaxPool2D((2,2) , strides = 2 , padding = 'same')(norm_2)

#block 3
conv_3 = tf.keras.layers.Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu')(maxpool_2)
norm_3 = tf.keras.layers.BatchNormalization()(conv_3)
maxpool_3 = tf.keras.layers.MaxPool2D((2,2) , strides = 2 , padding = 'same')(norm_3)

#block 4
conv_4 = tf.keras.layers.Conv2D(128, (3,3), strides = 1, padding = 'same', activation = 'relu')(maxpool_3)
drop_2 = tf.keras.layers.Dropout(0.2)(conv_4)
norm_4 = tf.keras.layers.BatchNormalization()(drop_2)
maxpool_4 = tf.keras.layers.MaxPool2D((2,2), strides = 2, padding = 'same')(norm_4)

#block 5
conv_5 = tf.keras.layers.Conv2D(256,(3,3), strides = 1 , padding = 'same' , activation = 'relu')(maxpool_4)
drop_3 = tf.keras.layers.Dropout(0.2)(conv_5)
norm_5 = tf.keras.layers.BatchNormalization()(drop_3)
maxpool_5 = tf.keras.layers.MaxPool2D((2,2) , strides = 2 , padding = 'same')(norm_5)

#block 5
flatten = Flatten()(maxpool_5)

#Block 6
dense_1 = Dense(128, activation='relu')(flatten)
drop_3 = tf.keras.layers.Dropout(0.2)(dense_1)

#block 7
output = Dense(1, activation='sigmoid')(drop_3)

model = keras.Model(inputs=img_input, outputs=output)
model.compile(optimizer = "rmsprop" , loss = 'binary_crossentropy' , metrics = ['accuracy'])


In [30]:
cnn = model
cnn.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 150, 150, 1)]     0         
                                                                 
 conv2d_30 (Conv2D)          (None, 150, 150, 32)      320       
                                                                 
 batch_normalization_30 (Bat  (None, 150, 150, 32)     128       
 chNormalization)                                                
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 75, 75, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_31 (Conv2D)          (None, 75, 75, 64)        18496     
                                                                 
 dropout_24 (Dropout)        (None, 75, 75, 64)        0   

In [31]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.3, min_lr=0.000001)

In [32]:
history = cnn.fit(datagen.flow(x_train,y_train, batch_size = 32) ,epochs = 12 , validation_data = datagen.flow(x_val, y_val) ,callbacks = [learning_rate_reduction])

Epoch 1/12
163/163 [==============================] - ETA: 0s - loss: 0.5237 - accuracy: 0.8395

KeyboardInterrupt: 

In [13]:
cnn.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 150, 150, 32)      320       
                                                                 
 batch_normalization (BatchN  (None, 150, 150, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 dropout (Dropout)           (None, 75, 75, 64)        0     

In [3]:
import pós
model = pós.model()